<a href="https://colab.research.google.com/github/sxmepluie/Audio-AI-Transformer-Comparison/blob/main/audio_classification_for_wavlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
import pandas as pd

# Google Drive'ı bağla
drive.mount('/content/drive')

# CSV dosyalarını yükle
df_ozellik = pd.read_csv('/content/drive/MyDrive/df_ozellik.csv')
df_temp = pd.read_csv('/content/drive/MyDrive/df_temp.csv')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Özellikler ve etiketleri ayır
X = df_ozellik.drop("duygu", axis=1).values
y = df_ozellik["duygu"].values

# Etiketleri one-hot encoding yap
encoder = OneHotEncoder()
y = encoder.fit_transform(y.reshape(-1, 1)).toarray()

# Veriyi eğitim ve test setlerine ayır
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForSequenceClassification
import torch

# WavLM modelini ve özellik çıkarıcıyı yükle
model_name = "microsoft/wavlm-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = WavLMForSequenceClassification.from_pretrained(model_name, num_labels=y.shape[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/373M [00:00<?, ?B/s]

Some weights of Data2VecAudioForSequenceClassification were not initialized from the model checkpoint at facebook/data2vec-audio-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(audio_arrays, max_length=16000):

    # Ses verilerini belirli bir uzunluğa sıfırlarla doldur veya kırp
    audio_arrays = [np.pad(x, (0, max(0, max_length - len(x)))) if len(x) < max_length else x[:max_length] for x in audio_arrays]

    # Ses verilerini ön işleme
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return inputs

# Eğitim ve test verilerini ön işleme
x_train_processed = preprocess_function(x_train, max_length=16000)
x_test_processed = preprocess_function(x_test, max_length=16000)

In [ ]:
from datasets import Dataset
import torch

# Veriyi Dataset formatına dönüştür
train_dataset = Dataset.from_dict({
    'input_values': x_train_processed['input_values'],
    'labels': y_train.argmax(axis=1)
})

test_dataset = Dataset.from_dict({
    'input_values': x_test_processed['input_values'],
    'labels': y_test.argmax(axis=1)
})

# Belleği temizle
torch.cuda.empty_cache()

<ipython-input-7-749c1a4a7347>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.inputs = torch.tensor(inputs, dtype=torch.float32)  # Eğer tensör değilse tensöre çevir


Train Dataset Size: 16292
Test Dataset Size: 4073
Input Example: tensor([-8.3404e+01,  7.5786e+00, -5.3037e-01,  ..., -4.9113e-03,
        -4.9113e-03, -4.9113e-03])
Label Example: 3


In [ ]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import torch
import time

# Bellek temizleme callback'i
class MemoryCleanupCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()  # Her epoch sonunda belleği temizle

# Eğitim argümanlarını ayarla
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,  # Batch boyutunu artır
    per_device_eval_batch_size=16,   # Batch boyutunu artır
    gradient_accumulation_steps=1,   # Gradient accumulation'ı kapat
    num_train_epochs=1,              # Epoch sayısını azalt
    learning_rate=1e-4,              # Learning rate'i artır
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    report_to="none",
)

# Trainer'ı başlat
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[MemoryCleanupCallback()],  # Callback ekle
)

# Modeli eğit (hata yakalama ile)
try:
    start_train_time = time.time()
    trainer.train()
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    print(f"Eğitim Süresi: {training_time} saniye")
except Exception as e:
    print(f"Eğitim sırasında hata oluştu: {e}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
10,5.650100
20,5.590500
30,5.642000
40,5.615400
50,5.627200


Step,Training Loss
10,5.650100
20,5.590500
30,5.642000
40,5.615400
50,5.627200
60,5.564700
70,5.582700
80,5.547900
90,5.547400
100,5.564200


Eğitim Süresi: 7577.769493103027 saniye


In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

# Custom Dataset sınıfı
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        # Veriyi al ve tensöre çevir
        self.inputs = torch.tensor(inputs, dtype=torch.float32)  # Eğer tensör değilse tensöre çevir
        self.labels = torch.tensor(labels, dtype=torch.long)    # Argmax işlemi sonrası uzun tipte olmalı

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_values': self.inputs[idx],  # 2 boyutlu tensör
            'labels': self.labels[idx]         # 1 boyutlu tensör
        }

# Veriyi işleyerek CustomDataset'e dönüştür
train_inputs = x_train_processed['input_values']  # Verinin işlenmiş hali
train_labels = y_train.argmax(axis=1)  # Etiketlerin argmax ile alınması
train_dataset = CustomDataset(train_inputs, train_labels)

test_inputs = x_test_processed['input_values']  # Test verisi
test_labels = y_test.argmax(axis=1)  # Test etiketleri
test_dataset = CustomDataset(test_inputs, test_labels)

# Dataset'in boyutunu kontrol et
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")

# Örnek bir veri çekelim
example = train_dataset[0]
print(f"Input Example: {example['input_values']}")
print(f"Label Example: {example['labels']}")

# Modeli değerlendir
start_evaluate_time = time.time()
eval_results = trainer.evaluate()
end_evaluate_time = time.time()
evaluate_time = end_evaluate_time - start_evaluate_time

# Metrikleri hesapla
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
labels = test_dataset.labels  # 'labels' özelliğine doğru şekilde eriş

# Olasılık skorlarını hesapla
probabilities = softmax(predictions.predictions, axis=1)

# Metrikleri hesapla
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds, average='weighted', zero_division=0)  # UndefinedMetricWarning'ı önle
recall = recall_score(labels, preds, average='weighted')
f1 = f1_score(labels, preds, average='weighted')
auc = roc_auc_score(labels, probabilities, multi_class='ovr', average='weighted')  # Olasılık skorlarını kullan
cm = confusion_matrix(labels, preds)
sensitivity = np.diag(cm) / np.sum(cm, axis=1)
specificity = np.diag(cm) / np.sum(cm, axis=0)

# Sonuçları yazdır
print(f"Doğruluk: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC: {auc}")
print(f"Sensitivity: {np.mean(sensitivity)}")
print(f"Specificity: {np.mean(specificity)}")
print(f"Değerlendirme Süresi: {evaluate_time} saniye")

Doğruluk: 0.25460348637368035
Precision: 0.06482293527363282
Recall: 0.25460348637368035
F1 Score: 0.10333613126008083
AUC: 0.5004316231524892
Sensitivity: 0.25
Specificity: nan
Değerlendirme Süresi: 1184.8547549247742 saniye


<ipython-input-11-775f56d0a026>:39: RuntimeWarning: invalid value encountered in divide
  specificity = np.diag(cm) / np.sum(cm, axis=0)


In [ ]:
# Sonuçları DataFrame'e dönüştür
metrics = {
    "Model": ["WavLM"],
    "Training Time": [training_time],
    "Evaluate Time": [evaluate_time],
    "Accuracy": [accuracy],
    "Precision": [precision],
    "Recall": [recall],
    "F1 Score": [f1],
    "AUC": [auc],
    "Sensitivity": [np.mean(sensitivity)],
    "Specificity": [np.mean(specificity)]
}

results_df = pd.DataFrame(metrics)
results_df.to_csv('/content/drive/MyDrive/Colab-Notebooks/results_wavlm.csv', index=False)
print(results_df)

      Model  Training Time  Evaluate Time  Accuracy  Precision    Recall  \
0  Data2Vec    7577.769493    1184.854755  0.254603   0.064823  0.254603   

   F1 Score       AUC  Sensitivity  Specificity  
0  0.103336  0.500432         0.25          NaN  


In [ ]:
import torch

# Belleği temizle
torch.cuda.empty_cache()